In [1]:
import mediapipe as mp
import cv2
from dollarpy import Recognizer, Template, Point
import pickle

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

gesture_templates = {
    "OneFinger": ["./vids/OneFinger1.mp4", "./vids/OneFinger2.mp4", "./vids/OneFinger3.mp4"],
    "TwoFingers": ["./vids/TwoFingers1.mp4", "./vids/TwoFingers2.mp4", "./vids/TwoFingers3.mp4"],
    "ThreeFingers": ["./vids/ThreeFingers1.mp4", "./vids/ThreeFingers2.mp4", "./vids/ThreeFingers3.mp4", "./vids/ThreeFingers4.mp4", "./vids/ThreeFingers5.mp4"]
}

In [3]:
def capture_gesture_points(video_files, gesture_type):
    points = []
    for video_path in video_files:
        cap = cv2.VideoCapture(video_path)
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)

                if results.right_hand_landmarks or results.left_hand_landmarks:
                    hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks
                    index_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                    middle_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP]
                    ring_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_TIP]
                    pinky_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_TIP]

                    if gesture_type == "OneFinger":
                        if (index_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP].y and
                            middle_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP].y and
                            ring_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_MCP].y and
                            pinky_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_MCP].y):
                            points.append(Point(index_tip.x, index_tip.y, 1))

                    elif gesture_type == "TwoFingers":
                        if (index_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP].y and
                            middle_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP].y and
                            ring_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_MCP].y and
                            pinky_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_MCP].y):
                            points.extend([Point(index_tip.x, index_tip.y, 1), Point(middle_tip.x, middle_tip.y, 1)])

                    elif gesture_type == "ThreeFingers":
                        if (index_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP].y and
                            middle_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP].y and
                            ring_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_MCP].y and
                            pinky_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_MCP].y):
                            points.extend([Point(index_tip.x, index_tip.y, 1), Point(middle_tip.x, middle_tip.y, 1), Point(ring_tip.x, ring_tip.y, 1)])

                cv2.imshow("Training Gesture", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()

    return points

In [4]:
templates = []
for gesture_type, video_paths in gesture_templates.items():
    points = capture_gesture_points(video_paths, gesture_type)
    if points:
        templates.append(Template(gesture_type, points))


In [5]:
model_filename = 'gesture_templates.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(templates, model_file)
print(f"Model saved to {model_filename}")

Model saved to gesture_templates.pkl


In [6]:
# Load the model
model_filename = 'gesture_templates.pkl'
with open(model_filename, 'rb') as model_file:
    templates = pickle.load(model_file)
    

recognizer = Recognizer(templates)
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:

def detect_gestures_real_time():
    cap = cv2.VideoCapture(0)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            detected_gesture = "No gesture"
            if results.right_hand_landmarks or results.left_hand_landmarks:
                hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks
                points = []

                # Get fingertip landmarks
                index_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                middle_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_TIP]

                # Draw circles on specific fingertip points
                def draw_circle(landmark, color):
                    cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                    cv2.circle(image, (cx, cy), 10, color, -1)

                # Detect gestures and show points for each gesture
                if (index_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP].y and
                    middle_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP].y and
                    ring_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_MCP].y and
                    pinky_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_MCP].y):
                    points.append(Point(index_tip.x, index_tip.y, 1))
                    detected_gesture = "One Finger Up"
                    draw_circle(index_tip, (0, 255, 0))  # Green circle for One Finger Up

                elif (index_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP].y and
                      middle_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP].y and
                      ring_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_MCP].y and
                      pinky_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_MCP].y):
                    points.extend([Point(index_tip.x, index_tip.y, 1), Point(middle_tip.x, middle_tip.y, 1)])
                    detected_gesture = "Two Fingers Up"
                    draw_circle(index_tip, (0, 0, 255))     # Red circle for index finger
                    draw_circle(middle_tip, (255, 0, 0))   # Blue circle for middle finger

                elif (index_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP].y and
                      middle_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP].y and
                      ring_tip.y < hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_MCP].y and
                      pinky_tip.y > hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_MCP].y):
                    points.extend([Point(index_tip.x, index_tip.y, 1), Point(middle_tip.x, middle_tip.y, 1), Point(ring_tip.x, ring_tip.y, 1)])
                    detected_gesture = "Three Fingers Up"
                    draw_circle(index_tip, (255, 255, 0))  # Cyan for index
                    draw_circle(middle_tip, (0, 255, 255)) # Yellow for middle
                    draw_circle(ring_tip, (255, 0, 255))   # Magenta for ring

            cv2.putText(image, f"Gesture: {detected_gesture}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

           
            cv2.imshow("Gesture Detection", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
detect_gestures_real_time()